In [1]:
import random
from deap import base, creator, tools, algorithms

# 定义目标函数
def fitness(individual):
    x1, x2, x3 = individual
    return 4*x1 + 9*x2 + 6*x3

# 定义约束条件
def const1(individual):
    x1, x2, x3 = individual
    return x1 - x3 - 11.23 <= 0

def const2(individual):
    x1, x2, x3 = individual
    return x2 + x3 - 12 == 0

def const3(individual):
    x1, x2, x3 = individual
    return x1 + 2*x2 - 1.5 >= 0


# 定义适应度函数和约束惩罚函数
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("attr_x1", random.uniform, 0, 10)
toolbox.register("attr_x2", random.uniform, 0, 10)
toolbox.register("attr_x3", random.uniform, 0, 10)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_x1, toolbox.attr_x2, toolbox.attr_x3), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness)

# 定义约束函数和约束惩罚函数
toolbox.decorate("evaluate", tools.DeltaPenalty(const1, 100.0))
toolbox.decorate("evaluate", tools.DeltaPenalty(const2, 100.0))
toolbox.decorate("evaluate", tools.DeltaPenalty(const3, 100.0))

# 定义遗传算法操作
toolbox.register("mate", tools.cxSimulatedBinary, eta=3.0)
toolbox.register("mutate", tools.mutPolynomialBounded, low=[0, -100, -100], up=[10, 100, 100], eta=3.0, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# 定义种群和进化代数
population_size = 50
num_generations = 100

# 生成初始种群并进行进化
population = toolbox.population(n=population_size)
for generation in range(num_generations):
    offspring = algorithms.varOr(population, toolbox, lambda_=population_size, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=population_size)

# 输出最优解
best_ind = tools.selBest(population, k=1)[0]
print("Best individual is: %s\nwith fitness: %s" % (best_ind, best_ind.fitness.values[0]))


Best individual is: [2.894195125407783, -10.025430302563741, -19.70556800861928]
with fitness: 100.0


In [2]:
print(fitness(best_ind))

-196.88550027315821
